In [1]:
import os
from os import path
import sqlite3
global_sqlite_filename='apf.db'
import pandas
from datetime import datetime, timedelta
import numpy as np
from astropy.time import Time
from sqlalchemy import create_engine
import pytz
import numpy as np

In [2]:
class col_types:
    def __init__(self):
        self.TEXT = 'TEXT'
        self.INTEGER = 'INTEGER'
        self.REAL = 'REAL'
        self.BLOB = 'BLOB'
        self.NULL = 'NULL'
        
ct = col_types()
print(ct.TEXT)

#print(os.getcwd())
#print(os.listdir())

TEXT


In [3]:
def create_database(sqlite_filename=global_sqlite_filename):
    conn = sqlite3.connect(sqlite_filename)
    cursor = conn.cursor()
    return cursor
       
def get_telemetry_columns():
    columns = {'DateTime': ct.TEXT, 
                  'TARGET': ct.NULL,
                  'MIDPTFIN': ct.REAL,
                  'AZ': ct.REAL,
                  'EL': ct.REAL,
                  'AZENCPOS': ct.REAL,
                  'ELENCPOS': ct.REAL,
                  'AZENCVEL': ct.REAL,
                  'ELENCVEL': ct.REAL,
                  'AZFLWERR': ct.REAL,
                  'ELFLWERR': ct.REAL,
                  'OUTFILE': ct.REAL,
                  'OBSNUM': ct.REAL,
                  'MODE': ct.REAL,
                  'AVG_FWHM': ct.REAL,
                  'M5WIND': ct.REAL,
                  'M5WINDAZ': ct.REAL,
                  'TAVERAGE': ct.REAL,
                  'TM1S210': ct.REAL,
                  'TM2CAIR': ct.REAL,
                  'OFFSET_AZ': ct.REAL,
                  'OFFSET_EL': ct.REAL,
                  'RMSOFFSET_AZ': ct.REAL,
                  'RMSOFFSET_EL': ct.REAL,
                  'AVGOFFSET_AZ': ct.REAL,
                  'AVGOFFSET_EL': ct.REAL,
                  'HATCHPOS': ct.TEXT,
                  'EVENT': ct.TEXT}    
    return columns
    
def get_velocity_columns():
    columns = [('MdptJulian', ct.REAL),
               ('velocity', ct.REAL),
               ('velocity_error', ct.REAL),
               ('stellar_activity1', ct.REAL),
               ('stellar_activity2', ct.REAL),
               ('total_counts', ct.REAL),
               ('brad_doesnt_know', ct.REAL),
               ('DateTimeUTC', ct.TEXT),
               ('DateTimeLocal', ct.TEXT),
               ('starID', ct.TEXT)
              ]
    return columns
    
def create_telemetry_table(cursor):        
    columns = get_telemetry_columns()
    l = ['{} {}'.format(k,v) for (k, v) in columns.items()]
    print(len(l))
    full_list = ', '.join(l)
    sql_create_string = 'CREATE TABLE telemetry (' + full_list + ', PRIMARY KEY (DateTime))'
    print(sql_create_string)        
    cursor.execute(sql_create_string)

    return


def create_velocity_table(cursor):
    columns = get_velocity_columns()
    l = ['{} {}'.format(k,v) for (k, v) in columns]
    full_list = ', '.join(l)
    sql_create_string = 'CREATE TABLE velocity (' + full_list + ', PRIMARY KEY (DateTimeLocal))'
    print(sql_create_string)  
    cursor.execute(sql_create_string)

    return

def open_connection(sqlite_filename=global_sqlite_filename):
    dburi = 'file:{}?mode=rw'.format(sqlite_filename)
    conn = sqlite3.connect(dburi, uri=True)
    return conn
    

In [4]:
conn = open_connection()
cursor = conn.cursor()
#create_telemetry_table(cursor)
#create_velocity_table(cursor)

In [5]:
def populate_telemetry_table(conn, filenames):
    for csv_file in filenames:
        try:
            df = pandas.read_csv(csv_file, 
                               sep='\t', skiprows=1, header=None, names=get_telemetry_columns(), comment='#')
            df.to_sql('telemetry', conn, if_exists='append', index=False)
            
        except Exception as e:
            print(csv_file)
            print(e)

def populate_telemetry(conn=None, dirname=None, batchsize=5):
    filenames = [os.path.join(dirname, x) for x in os.listdir(dirname)]
    filenames.sort()
    
    def batch(iterable, n=1):
        l = len(iterable)
        for ndx in range(0, l, n):
            yield iterable[ndx : min(ndx + n, l)]
    
    for file_batch in batch(filenames, n=batchsize):
        populate_telemetry_table(conn, file_batch)
        
    


        

In [6]:
#populate_telemetry(conn, '../telemetry_data', batchsize=10)

In [7]:
def load_velocity_csv(dirname, filename):
    full_path = os.path.join(dirname, filename)
    columns = get_velocity_columns()[:-2] # leave off the last two column names
                                        # because we're calculating them below.
    df = pandas.read_csv(full_path, sep='\s+',
            names=[x[0] for x in columns]) 
                                                
    return df
    
    
california_tzinfo = pytz.timezone('US/Pacific')

# returns a string for a single julian date. Couldn't figure out how to vectorize.
def convert_julian_to_datetime_local(juliantime):
    times = Time(juliantime, format='jd')
    full_iso = times.to_datetime(timezone=california_tzinfo).isoformat()
    return full_iso[:-6]

# takes in an array of julian dates. Returns an array of local isot strings.
def make_array_of_local_datetimes(julian_array):
    output_array = output_array = np.chararray(len(julian_array), itemsize=26, unicode=True)
    for i, julian_time in enumerate(julian_array):
        output_array[i] = convert_julian_to_datetime_local(julian_time)
    return output_array
    

def convert_julian_to_datetime_utc(juliantimes):
    times = Time(juliantimes, format='jd')
    return times.isot 
    
def add_calculated_columns(df, filename):
    new_df = df.copy()
    #datetimes = convert_julian_to_datetime(new_df.MdptJulian)
    new_df['DateTimeUTC'] = convert_julian_to_datetime_utc(new_df.MdptJulian)
    new_df['DateTimeLocal'] = make_array_of_local_datetimes(new_df.MdptJulian)
    new_df['starID'] = [filename] * len(new_df.index) 
    
    return new_df
    

def populate_velocity(conn, dirname='../standard_star_velocity_measurements/', debug=False):
    filenames = ['HD10700_APF.vels', 'HD185144_APF.vels', 'HD9407_APF.vels']
    for file in filenames:
        print('started {}'.format(file))
        df_incomplete = load_velocity_csv(dirname, file)
        df_complete = add_calculated_columns(df_incomplete, file)
        if debug:
            print(df_complete)
        else: 
            df_complete.to_sql('velocity', conn, if_exists='append', index=False)
        print('tried to add {} entries to \'velocity\''.format(len(df_complete)))

#velocity_df = load_velocity_csv('../standard_star_velocity_measurements/', 'HD10700_APF.vels')
#add_calculated_columns(velocity_df, 'HD10700_APF.vels' )


In [9]:
#populate_velocity(conn)

started HD10700_APF.vels
tried to add 755 entries to 'velocity'
started HD185144_APF.vels
tried to add 1655 entries to 'velocity'
started HD9407_APF.vels
tried to add 694 entries to 'velocity'


In [8]:
# create additional calculated columns on velocity
engine = create_engine('sqlite:///apf.db', echo=False)


In [9]:
#add new table telemetrytimelookup
def drop_telemetry_time_lookup(engine):
    engine.execute("""DROP TABLE IF EXISTS telemetry_time_lookup;""")
    print('dropped telemetry_time_lookup.')

def create_telemetry_time_lookup(engine):
    engine.execute("""
    CREATE TABLE telemetry_time_lookup (
    DateTime TEXT PRIMARY KEY,
    YearMonthDayHour REAL,
    EpocSeconds REAL,
    Event TEXT,
    ExposureID TEXT,
    FOREIGN KEY (DateTime) REFERENCES telemetry(DateTime) 
    )""")
    print('created telemetry_time_lookup.')

def populate_telemetry_time_lookup(engine):   
    engine.execute("""INSERT INTO telemetry_time_lookup (DateTime,
        YearMonthDayHour, EpocSeconds, Event) 
    SELECT t.DateTime, strftime('%Y-%m-%d-%H', t.datetime), strftime('%s', t.DateTime), t.Event
    from telemetry t
    """)
    print('populated created telemetry_time_lookup.')
    

def create_index_in_telemetry_time_lookup(engine):
    engine.execute("""CREATE INDEX idx_yearmonthdayhour ON 
        telemetry_time_lookup(YearMonthDayHour, EpocSeconds)
    """)
    print('created index on YearMonthDayHour in TTC.')
    
#drop_telemetry_time_lookup(engine)
#create_telemetry_time_lookup(engine)
#populate_telemetry_time_lookup(engine)
#create_index_in_telemetry_time_lookup(engine)

df = pandas.read_sql_query("""select epocseconds - 1436589861.0, event from (select * from telemetry_time_lookup
    where YearMonthDayHour = '2015-07-11-04') 
    where event != 'ExposureBegin' 
    and epocseconds - 1436589861.0 < 0
    order by  abs(1436589861.0 - epocseconds) 
    limit 1
    """, con=engine)
pandas.set_option('precision', 18)
print(df)



   epocseconds - 1436589861.0       Event
0                       -38.0  EraseBegin


In [118]:
def drop_vel_telemetry_stats(engine):
    engine.execute("""DROP TABLE IF EXISTS vel_telemetry_stats;""")
    print('dropped vel_telemetry_stats.')

def create_vel_telemetry_stats(engine):
    engine.execute("""
    CREATE TABLE vel_telemetry_stats (
                  DateTimeLocal TEXT PRIMARY KEY,
                  YearMonthDayHour TEXT,
                  EpocSeconds REAL,
                  ExposureID TEXT,
                  exposure_begin_datetime TEXT,
                  exposure_end_datetime TEXT
                  EL_AVG REAL, EL_STD REAL,
                  AZENCPOS_AVG REAL, AZENCPOS_STD REAL
                  ELENCPOS_AVG REAL, ELENCPOS_STD REAL,
                  AZENCVEL_AVG REAL, AZENCVEL_STD REAL,
                  ELENCVEL_AVG REAL, ELENCVEL_STD REAL,
                  AZFLWERR_AVG REAL, AZFLWERR_STD REAL,
                  ELFLWERR_AVG REAL, ELFLWERR_STD REAL,
                  AVG_FWHM_AVG REAL, AVG_FWHM_STD REAL,
                  M5WIND_AVG REAL, M5WIND_STD REAL,
                  M5WINDAZ_AVG REAL, M5WINDAZ_STD REAL,
                  TAVERAGE_AVG REAL, TAVERAGE_STD REAL,
                  TM1S210_AVG REAL, TM1S210_STD REAL,
                  TM2CAIR_AVG REAL, TM2CAIR_STD REAL,
                  OFFSET_AZ_AVG REAL, OFFSET_AZ_STD REAL,
                  OFFSET_EL_AVG REAL, OFFSET_EL_STD REAL,
                  RMSOFFSET_AZ_AVG REAL, RMSOFFSET_AZ_STD REAL,
                  RMSOFFSET_EL_AVG REAL, RMSOFFSET_EL_STD REAL,
                  AVGOFFSET_AZ_AVG REAL, AVGOFFSET_AZ_STD REAL,
                  AVGOFFSET_EL_AVG REAL, AVGOFFSET_EL_STD REAL,
                  Delta_temp_from_begin_night REAL, 
    FOREIGN KEY (DateTimeLocal) REFERENCES velocity(DateTimeLocal) 
    )""")
    print('created vel_telemetry_stats')
    return

def get_begin_end_exposure_datetimes(YearMonthDayHour, epocseconds, engine):
    df_begin = pandas.read_sql_query("""select epocseconds - {}, event from (select * from telemetry_time_lookup
    where YearMonthDayHour = {1}) 
    where event != 'ExposureBegin' 
    and epocseconds - {0} < 0
    order by  abs({0} - epocseconds) 
    limit 1
    """.format(), con=engine)
    
    df_end = pandas.read_sql_query("""select epocseconds - {}, event from (select * from telemetry_time_lookup
    where YearMonthDayHour = {1}) 
    where event != 'ExposureBegin' 
    and epocseconds - {0} > 0
    order by  abs({0} - epocseconds) 
    limit 1
    """.format(), con=engine)
    
    return df_begin, df_end

def populate_only_datetime_vts(engine):
    engine.execute("""Insert into vel_telemetry_stats
                                    (DateTimeLocal, YearMonthDayHour, EpocSeconds)
                                    select v.DateTimeLocal, 
                                        strftime('%Y-%m-%d-%H', v.DateTimeLocal), 
                                        strftime('%s', v.DateTimeLocal)
                                    from velocity v
                                    where v.DateTimeLocal > "2015-01-01T17:00:00" """)
    print(pandas.read_sql_query("""select * from vel_telemetry_stats order by datetimelocal desc limit 10""", engine))
    
def get_begin_end_times(engine, VelocityDateTime, printstuff=False):
    DateTime = VelocityDateTime
    if printstuff:
        print('starting datetime: {}'.format(DateTime))
        
    def make_sql(DateTime, TrueForBefore=True):
        if TrueForBefore:
            greaterthan = '<'
            ordering = 'desc'
        else:
            greaterthan = '>'
            ordering = 'asc'
        output = """select datetime, epocseconds - strftime("%s", "{0}"), event from 
            (select * from telemetry_time_lookup where 
            YearMonthDayHour =  strftime('%Y-%m-%d-%H', "{0}" ) 
        union all
            select * from telemetry_time_lookup where 
            YearMonthDayHour = (select yearmonthdayhour from telemetry_time_lookup 
                where YearMonthDayHour {1} strftime('%Y-%m-%d-%H', "{0}") order by 
                YearMonthDayHour {2} limit 1)
            ) 
         where event != 'ExposureBegin' 
            and epocseconds - strftime("%s", "{0}") {1} 0
            order by  abs(strftime("%s", "{0}") - epocseconds) 
            limit 1""".format(DateTime, greaterthan, ordering)
        
        return output
    
    if printstuff:
        print(make_sql(VelocityDateTime, TrueForBefore=True))
        print(make_sql(VelocityDateTime, TrueForBefore=False))
    
    df_begin = pandas.read_sql_query(make_sql(VelocityDateTime, TrueForBefore=True), engine)
    df_begin1 = pandas.read_sql_query("""select datetime, epocseconds - strftime("%s", "{0}"), event from 
            (select * from telemetry_time_lookup where 
            YearMonthDayHour =  strftime('%Y-%m-%d-%H', "{0}" ) 
        union all
            select * from telemetry_time_lookup where 
            YearMonthDayHour = (select yearmonthdayhour from telemetry_time_lookup 
                where YearMonthDayHour < strftime('%Y-%m-%d-%H', "{0}") order by 
                YearMonthDayHour desc limit 1)
            ) 
         where event != 'ExposureBegin' 
            and epocseconds - strftime("%s", "{0}") < 0
            order by  abs(strftime("%s", "{0}") - epocseconds) 
            limit 1
    """.format(DateTime), engine) 
    
    df_end = pandas.read_sql_query(make_sql(VelocityDateTime, TrueForBefore=False), engine)
    df_end1 = pandas.read_sql_query("""select datetime, epocseconds - strftime("%s", "{0}"), event from 
        (select * from telemetry_time_lookup where 
        YearMonthDayHour =  strftime('%Y-%m-%d-%H', "{0}" )) 
        where event != 'ExposureBegin' 
        and epocseconds - strftime("%s", "{0}") > 0
        order by  abs(strftime("%s", "{0}") - epocseconds) 
        limit 1
    """.format(DateTime), engine) 
    
    if printstuff:
        print('just prior time: {}'.format(df_begin))
        print(df_end)
    
    no_telemetry_message = 'no telemetry for this time'
    if df_begin.empty:
        first_timestamp = no_telemetry_message
        #raise Exception('df_begin empty for datetimelocal= {}'.format(VelocityDateTime))
    else:
        just_prior_timestamp = df_begin.DateTime[0]
        first_timestamp =  pandas.read_sql_query("""
            select DateTime from telemetry_time_lookup where
            datetime > "{}" limit 1
            """.format(just_prior_timestamp), engine).DateTime[0]
        
    if df_end.empty:
        last_timestamp = no_telemetry_message
        #raise Exception('df_end empty for datetimelocal= {}'.format(VelocityDateTime))
    else:
        just_after_timestamp = df_end.DateTime[0]
        last_timestamp =  pandas.read_sql_query("""
            select DateTime from telemetry_time_lookup where
            datetime < "{}" order by DateTime desc limit 1
            """.format(just_after_timestamp), engine).DateTime[0]
        
    
    
    
    
    
    if printstuff:
        print(just_prior_timestamp, just_after_timestamp)
        print(first_timestamp, last_timestamp)
    
    return first_timestamp, last_timestamp
    

def populate_begin_end_times(engine, printstuff=False):
    
    
    velocity_times = pandas.read_sql_query("""
               select v.datetimelocal from velocity v
                 inner join vel_telemetry_stats vt
                 on v.datetimelocal = vt.datetimelocal
                 where v.datetimelocal > "2015-01-01T00:00:58.463989"
                 and vt.exposure_begin_datetime is null
    """, engine).DateTimeLocal 
    
    for v_timestamp in velocity_times:
        if printstuff:
            print('trying {}'.format(v_timestamp))
        begin_time, end_time = get_begin_end_times(engine, v_timestamp)
        if printstuff:
            print('{} {}'.format(begin_time, end_time))
        engine.execute("""UPDATE vel_telemetry_stats 
                            SET exposure_begin_datetime = "{exposure_begin}",
                                exposure_end_datetime = "{exposure_end}"
                            WHERE DateTimeLocal = "{v_timestamp}"
                            """.format(exposure_begin=begin_time, 
                                      exposure_end=end_time,
                                      v_timestamp=v_timestamp))
    
    print("just tried to populate {} rows of begin-end timestamps".format(len(velocity_times)))

#drop_vel_telemetry_stats(engine)
#create_vel_telemetry_stats(engine)
#populate_only_datetime_vts(engine)

populate_begin_end_times(engine, printstuff=True) #this takes a while.



#get_begin_end_times(engine, VelocityDateTime='2015-07-20T04:00:58.463989', printstuff=False)
#get_begin_end_times(engine, VelocityDateTime='2015-07-25T03:55:50.016014')



trying 2017-01-31T21:40:17.184006
no telemetry for this time 2017-01-31T06:59:59.0187
trying 2017-01-31T22:05:37.823997
no telemetry for this time 2017-01-31T06:59:59.0187
trying 2017-01-31T22:25:00.768001
no telemetry for this time 2017-01-31T06:59:59.0187
trying 2017-03-17T20:38:08.736007
2017-03-17T20:27:39.0170 2017-03-17T20:47:29.0238
trying 2017-03-17T20:58:02.784006
2017-03-17T20:48:15.0185 2017-03-17T21:08:00.8703
trying 2017-04-03T05:36:37.151993
2017-04-03T05:26:06.0192 2017-04-03T05:46:05.0192
trying 2017-04-03T05:57:11.808020
2017-04-03T05:46:52.0172 2017-04-03T06:06:50.6034
trying 2017-06-04T03:07:12.288006
2017-06-04T02:57:24.1555 2017-06-04T03:17:23.7304
trying 2017-06-04T03:28:14.592002
2017-06-04T03:18:08.3040 2017-06-04T03:38:08.0164
trying 2017-06-21T04:58:37.920019
2017-06-21T04:48:45.0159 2017-06-21T05:08:43.0119
trying 2017-06-25T04:51:43.199999
2017-06-25T04:41:33.0142 2017-06-25T05:01:32.2863
trying 2017-06-30T04:34:16.895998
2017-06-30T04:24:07.5479 2017-06-30T

trying 2018-09-10T02:47:43.295996
2018-09-10T02:42:49.6988 2018-09-10T02:52:32.1132
trying 2018-09-11T22:52:35.039991
2018-09-11T22:48:01.5423 2018-09-11T22:56:41.4772
trying 2018-09-11T23:02:37.247990
2018-09-11T22:57:27.0277 2018-09-11T23:07:44.9722
trying 2018-09-11T23:13:38.207994
2018-09-11T23:08:31.0279 2018-09-11T23:18:24.3267
trying 2018-09-13T01:57:27.936009
2018-09-13T01:49:53.0254 2018-09-13T02:04:53.4341
trying 2018-09-13T02:12:44.639989
2018-09-13T02:05:38.7818 2018-09-13T02:19:43.0223
trying 2018-09-14T01:53:12.191993
2018-09-14T01:46:38.7362 2018-09-14T01:59:31.5597
trying 2018-09-14T02:06:51.263993
2018-09-14T02:00:18.0257 2018-09-14T02:13:32.0282
trying 2018-09-16T01:48:46.080018
2018-09-16T01:43:41.0228 2018-09-16T01:54:19.8965
trying 2018-09-16T02:00:44.063973
2018-09-16T01:55:06.0200 2018-09-16T02:06:22.0371
trying 2018-09-19T01:19:43.391985
2018-09-19T01:14:45.0232 2018-09-19T01:24:51.0246
trying 2018-09-19T01:30:53.856021
2018-09-19T01:25:37.0287 2018-09-19T01:36:

In [114]:
get_begin_end_times(engine, "2017-01-31T21:40:17.184006", printstuff=True)
    
    
    

starting datetime: 2017-01-31T21:40:17.184006
select datetime, epocseconds - strftime("%s", "2017-01-31T21:40:17.184006"), event from 
            (select * from telemetry_time_lookup where 
            YearMonthDayHour =  strftime('%Y-%m-%d-%H', "2017-01-31T21:40:17.184006" ) 
        union all
            select * from telemetry_time_lookup where 
            YearMonthDayHour = (select yearmonthdayhour from telemetry_time_lookup 
                where YearMonthDayHour < strftime('%Y-%m-%d-%H', "2017-01-31T21:40:17.184006") order by 
                YearMonthDayHour desc limit 1)
            ) 
         where event != 'ExposureBegin' 
            and epocseconds - strftime("%s", "2017-01-31T21:40:17.184006") < 0
            order by  abs(strftime("%s", "2017-01-31T21:40:17.184006") - epocseconds) 
            limit 1
select datetime, epocseconds - strftime("%s", "2017-01-31T21:40:17.184006"), event from 
            (select * from telemetry_time_lookup where 
            YearMonthDayH

Exception: df_begin empty for datetimelocal= 2017-01-31T21:40:17.184006

In [122]:
df = pandas.read_sql_query("""select v.datetimelocal from velocity v
                 inner join vel_telemetry_stats vt
                 on v.datetimelocal = vt.datetimelocal
                 where v.datetimelocal > "2015-01-01T00:00:58.463989"
                 and vt.exposure_begin_datetime = 'no telemetry for this time'""", engine)

#df = pandas.read_sql_query("""select datetime from telemetry_time_lookup where yearmonthdayhour='2017-01-31-21'
#                        
 #                       """, engine)

df

,DateTimeLocal
0,2017-01-31T21:40:17.184006
1,2017-01-31T22:05:37.823997
2,2017-01-31T22:25:00.768001


In [ ]:
df = pandas.read_sql_query(
"""SELECT
  event,
  COUNT(*) AS `num`
FROM
  (select * from telemetry where 
                        DateTime between "2015-02-02T17:00:00.0000" and 
                        "2015-02-03T05:01:00.0000") 
GROUP BY
  event                  
                        """, engine)

df


In [ ]:
df = pandas.read_sql_query(
"""SELECT DateTime, event from (select * from telemetry where DateTime < "2015-01-02T19:01:00.0000"
    and (event == "ExposureEnd" or event == 'ReadoutEnd')) order by datetime
limit 100
                        """, engine)

df


In [ ]:
#df1 = pandas.read_sql_query("""
#    Select Datetimelocal, Mdptjulian from velocity 
#    where DateTimeLocal > "2015-04-01T17:01:00.0000"
#    and DateTimeLocal < "2015-04-10T17:01:00.0000"
#    order by DateTimeLocal limit 100
#
#    """, engine)
#df2 = pandas.read_sql_query(
#"""select datetime from telemetry 
#    order by (abs(strftime('%s',datetime) - abs(strftime('%s',"2015-04-02T01:31:44.832007"))))  limit 1
#                        """, engine)

#print(df1)

print(df2)

#print(pandas.read_sql_query("""
#    Select t.DateTime, v.Datetimelocal, v.Mdptjulian from (velocity v, telemetry t)
#    where v.DateTimeLocal > "2015-04-01T17:01:00.0000"
#    and v.DateTimeLocal < "2015-04-10T17:01:00.0000"
#    and (t.DateTime - v.DateTi 
#    order by DateTimeLocal limit 100
#
#    """, engine))

In [106]:
"""a {0} b {0} c {0}""".format('hi')

'a hi b hi c hi'

In [122]:
from datetime import date

In [125]:
datestr = "2015-01-02T19:01:00.0000"
date.fromtimestamp(datestr)

TypeError: an integer is required (got type str)